In [1]:
import pandas as pd
import os

In [9]:
from soynlp.tokenizer import RegexTokenizer, LTokenizer, MaxScoreTokenizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
def okt_tokenizer(x):
    return Okt.morphs(x)
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [4]:
lecture = pd.read_csv('lecture106.csv')
detail = pd.read_csv('detail107.csv')
review = pd.read_csv('review107.csv')

In [60]:
print('강의의 수 : ', len(lecture))
print('리뷰있는 강의수 : ',len(lecture.query('score != 0')['link'].unique()))
print('리뷰의 수 : ', len(review))

강의의 수 :  42738
리뷰있는 강의수 :  3541
리뷰의 수 :  28326


In [14]:
from konlpy.tag import Twitter
okt = Okt()

def okt_tokenizer(x):
    return okt.morphs(x)

In [18]:
data = review.groupby(['name','prof'])['review'].agg(lambda x : ' '.join(x)).reset_index()

In [20]:
data['token'] = data['review'].apply(okt_tokenizer)

In [27]:
keys=[]
for n,f in zip(data['name'],data['prof']):
    keys.append('-'.join([n,f]))

In [30]:
data['keys'] = keys

In [31]:
from collections import namedtuple
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d,c) for d, c in data[['token', 'keys']].values]
print(len(tagged_train_docs))

3525


In [34]:
doc_vectorizer = Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=8,        # distance between the predicted word and context words
    size=100,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=10,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=cores)   # multi cpu
    #hs = 1,          # hierarchical softmax / default 0
    #negative = 10,   # negative sampling / default 5)

In [35]:
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

import time
from tqdm import tnrange
start_time = time.time()

for epoch in tnrange(10):
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

print("start_time", start_time) 
print("--- %s seconds ---" %(time.time() - start_time))

X = np.array([doc_vectorizer.infer_vector(doc.words) for doc in tagged_train_docs])
y = np.array([doc.tags for doc in tagged_train_docs])

Doc2Vec(dbow+w,d100,n5,w8,mc10,s0.001,t8)


C:\Users\kcg99\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if __name__ == '__main__':



start_time 1571213527.4398065
--- 456.6279067993164 seconds ---


ValueError: invalid literal for int() with base 10: '3D Digital Modeling-허도석'

In [65]:
data.query('prof == "조윤호"')

,name,prof,review,token,keys
984,데이터마이닝,조윤호,빅데이터를 복수전공할 계획이 있는 사람들은 이 수업을 꼭 듣는 것을 추천합니다 ...,"[빅데이터, 를, 복수, 전공, 할, 계획, 이, 있는, 사람, 들, 은, 이, 수...",데이터마이닝-조윤호
1431,분석프로그래밍I,조윤호,이 교수님이 왜 평이 좋은지 모르겠네 모르면 나 말고 구글한테 물어봐 그렇다고...,"[이, 교수, 님, 이, 왜, 평이, 좋은지, 모르겠네, 모르면, 나, 말고, 구글...",분석프로그래밍I-조윤호
1432,분석프로그래밍II,조윤호,고난의 연속입니다 하지만 이만큼 배워가는 수업 또한 없다고 생각합니다 정말 노...,"[고난, 의, 연속, 입니다, 하지만, 이, 만큼, 배워가는, 수업, 또한, 없다고...",분석프로그래밍II-조윤호


In [68]:
def show(idx):
    print(data.iloc[idx,]['name'])
    print(data.iloc[idx,]['prof'])
    print(data.iloc[idx,]['review'])

In [72]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

A_sparse = sparse.csr_matrix(X)
similarities = cosine_similarity(A_sparse)

idx_list = np.argsort(similarities[984,:])[::-1][1:4]
for sim_idx in idx_list:
    show(sim_idx)
    print('')
    print('-'*30)

기업가정신과창업
황보윤
교수님은 친절하시고 괜찮은데 창업에 관심 없으면 수업 들어보려해도 귀에 전혀 안들어옴                  즉  기업가 정신에 대해서 배우는 수업입니다  이름만 들어보면 창업을 준비하는 경영대 학생들에게만 의미있어 보입니다  사실 비즈니스 모델을 수립해보며 자신의 전공 분야에 대한 비젼도 생각하고 이에 따라 이타적으로 미래를 계획해나갈 수 있는 의미를 갖고 있기도 합니다  아이디어 발표를 통해 직접 팀을 구성하여 사업 계획서를 구체화 해보고 외부 교수를 초빙하여 수행하는 아이디어톤을 통해 전문가의 시선으로 평가받아볼 수 있는 자리 또한 존재합니다  아이디어가 기획되는 프로세스를 배우고 실현시켜보면서 우리 주변에 보이는 사물과 서비스가 기획되고 생산되는 절차 모델도 배울 수 있습니다  이 기회를 잘 활용할 준비가 되었다면 실용적 측면에서 알맹이가 가득한 교양 강좌 중 하나라고 생각합니다 개귀찮음 뭔가 창업에 관심이 많고 아이디어좋으면 할만할듯 배워가는 것은 많다   하지만 힘들다  진정 자신이 창업에 뜻이 있으면 신청하길 바란다  조모임 매우매우매우매우많음 조활동 빡세게해봤자 시험못보면 소용없음

------------------------------
탐색적데이터분석
정여진
 로 데이터 시각화 배움  매주 과제가 있긴 한데 항상 수업 뒤에 한시간은 과제할 시간으로 주시기 때문에 과제로 크게 고생 하진 않고  프로젝트가 학점에 큰 영향 끼치는듯 시험문제는 쉽고 오픈북인데 팀플 점수폭이 너무 넓어서 그냥 팀플로 갈리는 강의 힘들었음 데이터 분석 실무 이전에  시각화를 통해 데이터를 탐색하는 것을 배웁니다  시험이나 과제에 피드백이 부족한 경향이 있지만 개인적으로 질문하면 다 받아주십니다   의 시각화를 전반적으로 훑는 강의  과제나 시험에 대한 피드백이 있었으면 좋았을듯 교수님 정말 좋습니다   학점 채워주시구요  실습수업이지만 이전에 수업을 듣고 들었다면 무리 없습니다 

------------------------------
청소

In [74]:
idx_list = np.argsort(similarities[1431,:])[::-1][1:4]
for sim_idx in idx_list:
    show(sim_idx)
    print('')
    print('-'*30)

분석프로그래밍II
조윤호
고난의 연속입니다   하지만 이만큼 배워가는 수업 또한 없다고 생각합니다  정말 노력하는 만큼 정직하게 알게 되는 수업입니다  교수님의 강의력과 수업은 정말 재미있고 유익하고 최고입니다  내용자체는 괜찮은 듯 하나 역시나 막상 코딩은 어렵   많이 배울 수 있는 수업입니다  상당히 수업을 잘하심 그러나 속도가 아웃사이더 급이어서 나의 두뇌도 아웃사이더가 되어버림  남는게 많을 것 같음 그러나 내 시험지를 보는 교수님에게 죄송해짐 판다스  넘파이 등 파이썬 라이브러리를 통한 데이터 전처리와 시각화를 배우는 수업  중간이나 기말이나 과제나 하는 만큼 결과를 얻을 수 있을겁니다  옛날엔 점수 낮으면 그냥 에프 때리셨는데 요즘도 그러시나 모르겠네요  파이썬의판다스패키지를배워가는강의입니다 열의도넘치시고재밋지만 과제시험이너무괴리감이큽니다

------------------------------
탐색적데이터분석
정여진
 로 데이터 시각화 배움  매주 과제가 있긴 한데 항상 수업 뒤에 한시간은 과제할 시간으로 주시기 때문에 과제로 크게 고생 하진 않고  프로젝트가 학점에 큰 영향 끼치는듯 시험문제는 쉽고 오픈북인데 팀플 점수폭이 너무 넓어서 그냥 팀플로 갈리는 강의 힘들었음 데이터 분석 실무 이전에  시각화를 통해 데이터를 탐색하는 것을 배웁니다  시험이나 과제에 피드백이 부족한 경향이 있지만 개인적으로 질문하면 다 받아주십니다   의 시각화를 전반적으로 훑는 강의  과제나 시험에 대한 피드백이 있었으면 좋았을듯 교수님 정말 좋습니다   학점 채워주시구요  실습수업이지만 이전에 수업을 듣고 들었다면 무리 없습니다 

------------------------------
공학응용프로그래밍
이상헌
교수님이 되게 열심히 알려주시려고 노력함  기말고사 범위부터 어려워서 수업을되게 따라가기어려움 학점은 잘 채워주심 한챕터를 한주에 다 나가는정도 로 진도 빠르고 중간고사까지는 많이 안어려운데 기말고사 범위가 어려워 시간투자를 좀 해야한다 시험 점수확인은 따로

In [79]:
data.query('prof == "안현상"')

,name,prof,review,token,keys
1214,문명과세계화의도전,안현상,교수님 강의스타일도 지루하지 않고 재밌습니다 시험은 외우는 거 잘하시는 분 추천드...,"[교수, 님, 강의, 스타일, 도, 지루하지, 않고, 재밌습니다, 시험, 은, 외우...",문명과세계화의도전-안현상
1231,문화와정치행태,안현상,이런게 대학수업 엄청나게 배워감 대단하신분 중간기말 레포트임 학점 안나와...,"[이런게, 대학, 수업, 엄청나게, 배워, 감, 대단하, 신분, 중간, 기, 말, ...",문화와정치행태-안현상
1807,세계화와다문화주의,안현상,좋은 수업입니다 추천해요 부담도 없고 주제에 대해서 열심히 생각만 해도 성적이 ...,"[좋은, 수업, 입니다, 추천, 해요, 부담, 도, 없고, 주제, 에, 대해, 서,...",세계화와다문화주의-안현상
1952,시민사회와정치,안현상,교수님 너무 좋으셨어요 수업내용은 특성상 조금 추상적이라 어렵기도 했지만 도움되는...,"[교수, 님, 너무, 좋으셨어요, 수업, 내용, 은, 특성, 상, 조금, 추상, 적...",시민사회와정치-안현상
2808,정치학연구방법론,안현상,갓 더는 표현 못합니다 중간 기말 레포트 대체이고 수업방식은 주로 조별 토론...,"[갓, 더, 는, 표현, 못, 합니다, 중간, 기, 말, 레포트, 대체, 이고, 수...",정치학연구방법론-안현상


In [80]:
idx_list = np.argsort(similarities[1214,:])[::-1][1:4]
for sim_idx in idx_list:
    show(sim_idx)
    print('')
    print('-'*30)

문명과세계화의도전
안드레이 란코프
피피티 달달 외워서 답 다쓰고 출석 다 채웠는데 왜   인지 이해 안 감 원어강의인데 살다온사람이 많아서 토종 한국인은 공부하기 힘듭니다  발음을 도저히 알아들을수가 없었어요  장담하건데 국민대 최고의 꿀강의  미국에서 쫌 살다왔거나 영어 잘하는 사람은 꼭 수강하시길  수업은 잘 들으면 얻어가는게 많은데 대부분 한국학생들은 안듣고 다른수업 과제하고 폰만지거나 자고있음   일정도 공부하고 영작잘하니까   받음 추천   항상   분정도 늦게 들어오심 출석 잘 안 부름 팀플  과제 없음 시험 기말 한번 편한건 맞는데 얻어가는건 없는 수업    교수님 수업이 공통적으로 성적입력도 가장 늦고 기간 지나서 보류까지 받고 겨우 성적 입력 하심 특히 이번 기말에서는 시험지 검토를 한번도 안하셨는지 시험지에 단답형 문제 답이 적혀있었는데 그냥 나눠줬다가 학생이 말해줘서 알고 당일날 시험지 다시 뽑아 주셨음    핵교만 아니었어도 안 듣는건데 수업 열심히 들었는데 제가 워낙 세계사에 무지해서 비쁠 나왔네요              워드로 정리도 했는데   꿀강인건 맞는데 그냥 그랬음  교수님 수업도 거의 항상 늦게 오시고 배워가는건 많은데 그냥 그런 강의임 교수님이 정말 똑똑하신 분이라는 게 느껴지는 교수님 중 한분  외국인 교수님이시지만 한국어나 한자수준이 너무 뛰어나심  기말도 한번만 보기때문에 열심히 이해하고 외우면 문제 없음   중간시험 과제 팀플 없음 교수님 출장으로 휴강 가끔 있음 기말시험만 보는데 시험 문제도 찝어줌 찝어준 문제 답만    에서 찾아서 외워서 적으면 점수 잘 주심 하루 전에 공부해서   받을 정도는 아님 일주일은 달달 외워야 함  영어쓰시다 애들이 못 알아들으면 유창한 한국어로 설명해주시기도 함  한국어 한국인 수준으로 잘하심  출석도 가끔 부르셔서 가끔 빠져도 됨  완전 꿀강 꼭 들어야 함  하루 전에 공부해도 에이플 나옴 기말시험만 있음  중간  과제  팀플    외국인 학생 유학생 영어 잘하는 한국인 다수 존재 